Version 2.0
---
Adresses the iteration limit of the agent from version 1. Agent is now a SQL agent that generates the query for a SQlite database with the dataset stored in it.


Imports
---

In [1]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

from langchain_openai import AzureChatOpenAI
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import requests
import os
import sqlite3

Load a large language model
----------
Langchain makes it possible to easily switch LLMs. Llama 3 is used to show the data can be analysed with a locally running open-source model, but it is very slow. So to speed it up I also used o3-mini and gpt-4o-mini to show it works.

Load Llama3:


In [2]:
chosen_llm = ChatOllama(base_url='http://localhost:11434', model="llama3")

Load o3-mini (via Azure):

In [3]:
load_dotenv()

chosen_llm = AzureChatOpenAI(model ="o3-mini", api_version="2025-01-01-preview", azure_endpoint="https://56948-m9bdjgpg-eastus2.cognitiveservices.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2025-01-01-preview", api_key=os.environ.get("AZURE_OPENAI_API_KEY"))

Load gpt-4o-mini (via Azure)

In [4]:
load_dotenv()

chosen_llm = AzureChatOpenAI(model="gpt-4o-mini", api_version="2025-01-01-preview",
                             azure_endpoint="https://56948-m9bdjgpg-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview",
                             api_key=os.environ.get("AZURE_OPENAI_API_KEY"))

Create local SQlite database
---
The database is created by building a SQlite database and creating a "database engine" object that will be used later for creating the agent's toolkit.

In [5]:
con = sqlite3.connect("greenery.db", check_same_thread=False)
cur = con.cursor()

# Create neighborhoods table in database
try:
    cur.execute("CREATE TABLE neighborhoods(name varchar(255), greeneryPercentage float)")
    print("Table created successfully.")

except:
    cur.execute("DROP TABLE neighborhoods")
    cur.execute("CREATE TABLE neighborhoods(name varchar(255), greeneryPercentage float)")
    print("Table has been reset, because it already existed.")

# Create database engine object
engine = create_engine(
        "sqlite://",
        creator=lambda: con,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )
db = SQLDatabase(engine)

Table created successfully.


Extract properties and save them in the database
---
The dataset is retrieved via an API-request and is then stored in the SQlite database.

In [6]:
# Retrieve dataset
dataset = requests.get('https://data.rivm.nl/geo/ank/ows?service=WFS&request=GetFeature&typeName=rivm_2022_groenpercentage_kaart_per_buurt&propertyName=bu_naam,_mean&outputFormat=json').json()

# Extract properties and put them into a list
properties = []
for feature in dataset["features"]:
    property = feature["properties"]
    properties.append((property["bu_naam"], property["_mean"]))

# Save the items stored in the list into the database
i=0
while i < len(properties):
    cur.execute("INSERT INTO neighborhoods VALUES (?,?)", properties[i])
    i = i + 1
con.commit()

Create and run the agent
---
Here the toolkit is created that "gives" the database to the agent. It also creates the agent itself, what will analyse the database and create a query for it to answer the question when it is executed.


In [7]:
# Create toolkit:
toolkit = SQLDatabaseToolkit(db=db,llm=chosen_llm)

# Create agent
agent_executor = create_sql_agent(
    llm=chosen_llm,
    toolkit=toolkit,
    verbose=True
)

# Run the agent
input_text = "Using the neighborhoods table, find the greenery percentage of the neighborhood with name Heerewaardan. If you can't find anything, check if the user made a spelling error in the neighborhood name and check if you can find the neighborhood with the corrected name, this should be a neighborhood that has only one or two letters different."
response = agent_executor.invoke(input_text, handleParsingErrors=True)
print (response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  neighborhoodsI will now check the schema of the neighborhoods table to see the relevant columns for the query.  
Action: sql_db_schema  
Action Input: neighborhoods  
CREATE TABLE neighborhoods (
	name VARCHAR(255), 
	"greeneryPercentage" FLOAT
)

/*
3 rows from neighborhoods table:
name	greeneryPercentage
Binnenstad-Noord	7.720674674163046
De Schenge	55.968981056131604
Streukel-Holten-Genne	82.48907110614155
*/The neighborhood "Heerewaardan" doesn't appear to exist, but I should check if there's a similar name. I will look for neighborhoods that have names differing by one or two letters. 

I'll first query the existing neighborhood names to find possible matches.  

Action: sql_db_query  
Action Input: SELECT name FROM neighborhoods WHERE name LIKE '%eer%' OR name LIKE '%waardan%';  [('Oostelbeers',), ('Bedrijventerrein Middenmeer',), ("'s-Heerenberg",), ('Verspreide huizen Berg en Nederweerte